In [1]:
def pretokenize(text):
    """
    Pretokenize the input text by splitting it into words and punctuation.
    """
    import re
    # Define a regex pattern to match words and punctuation
    pattern = r"\w+|[^\w\s]"
    # Find all matches in the input text
    tokens = re.findall(pattern, text)
    return tokens